In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('nhl_stats.csv', index_col=0)
df=df.dropna()
df['opponent'] = df.date.apply(lambda x: x.split()[1])

abb_to_team = {'PIT':'Pittsburgh Penguins', 'PHI':'Philadelphia Flyers',
'TOR':'Toronto Maple Leafs', 'MTL':'Montréal Canadiens', 'MIN':'Minnesota Wild',
'COL':'Colorado Avalanche', 'EDM':'Edmonton Oilers', 'CHI':'Chicago Blackhawks',
'CGY':'Calgary Flames', 'CAR':'Carolina Hurricanes', 'DAL':'Dallas Stars',
'OTT':'Ottawa Senators', 'BUF':'Buffalo Sabres', 'WSH':'Washington Capitals',
'ATL':'Atlanta Thrashers', 'ANA':'Anaheim Ducks', 'DET':'Detroit Red Wings',
'NJD':'New Jersey Devils','CBJ':'Columbus Blue Jackets', 'SJS':'San Jose Sharks',
'STL':'St. Louis Blues', 'PHX':'Phoenix Coyotes', 'NYR':'New York Rangers',
'TBL':'Tampa Bay Lightning', 'LAK':'Los Angeles Kings', 'NSH':'Nashville Predators',
'NYI':'New York Islanders', 'VAN':'Vancouver Canucks', 'BOS':'Boston Bruins',
'FLA':'Florida Panthers', 'WPG':'Winnipeg Jets', 'ARI':'Arizona Coyotes',
'VGK':'Vegas Golden Knights', 'SEA':'Seattle Kraken'}

df['identifier'] = df['date'].apply(lambda x: x.split()[0][10:])
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x[0:10], format='%Y/%m/%d'))
df['opponent'] = df['opponent'].apply(lambda x: abb_to_team[x])
df.head()

,team,date,points,RW,ROW,SOW,goals,goals_against,power_play,penalty_kill,net_ppp,net_pkp,shots,shots_against,FOWp,opponent,identifier
0,New Jersey Devils,2005-10-05,2,1,1,0,5,1,25.0,90.9,25.0,90.9,37.0,37.0,57.1,Pittsburgh Penguins,vs
1,New York Islanders,2005-10-05,0,0,0,0,4,6,14.3,81.8,14.3,81.8,26.0,39.0,51.5,Buffalo Sabres,@
2,New York Rangers,2005-10-05,2,1,1,0,5,3,50.0,90.0,50.0,90.0,27.0,27.0,51.6,Philadelphia Flyers,@
3,Philadelphia Flyers,2005-10-05,0,0,0,0,3,5,10.0,50.0,10.0,50.0,27.0,27.0,48.4,New York Rangers,vs
4,Pittsburgh Penguins,2005-10-05,0,0,0,0,1,5,9.1,75.0,9.1,75.0,37.0,37.0,42.9,New Jersey Devils,@


In [3]:
df = df.astype({'goals':int, 'goals_against':int, 'shots':float, 'shots_against':float, 'FOWp':float})
df.drop(columns=['RW', 'ROW', 'SOW', 'net_ppp', 'net_pkp'], inplace=True)
df.head()

,team,date,points,goals,goals_against,power_play,penalty_kill,shots,shots_against,FOWp,opponent,identifier
0,New Jersey Devils,2005-10-05,2,5,1,25.0,90.9,37.0,37.0,57.1,Pittsburgh Penguins,vs
1,New York Islanders,2005-10-05,0,4,6,14.3,81.8,26.0,39.0,51.5,Buffalo Sabres,@
2,New York Rangers,2005-10-05,2,5,3,50.0,90.0,27.0,27.0,51.6,Philadelphia Flyers,@
3,Philadelphia Flyers,2005-10-05,0,3,5,10.0,50.0,27.0,27.0,48.4,New York Rangers,vs
4,Pittsburgh Penguins,2005-10-05,0,1,5,9.1,75.0,37.0,37.0,42.9,New Jersey Devils,@


In [4]:
try:
    df['penalty_kill'] = pd.to_numeric(df['penalty_kill'])
except ValueError:
    mask = pd.to_numeric(df['penalty_kill'], errors='coerce').isna()
    L = df.loc[mask, 'penalty_kill'].tolist()
    print("Not converted values are: " + ", ".join(L))

Not converted values are: --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --

In [5]:
df['penalty_kill'].replace('--', 80.0, inplace=True)
df['power_play'].replace('--', 20.0, inplace=True)
df['penalty_kill'] = pd.to_numeric(df.penalty_kill)
df['power_play'] = pd.to_numeric(df.power_play)
df.dtypes

team                     object
date             datetime64[ns]
points                    int64
goals                     int32
goals_against             int32
power_play              float64
penalty_kill            float64
shots                   float64
shots_against           float64
FOWp                    float64
opponent                 object
identifier               object
dtype: object

In [6]:
df_home = df[df['identifier'] == 'vs']
df_home.drop(columns=['identifier'], inplace=True)
df_away = df[df['identifier'] == '@']
df_away.drop(columns=['identifier'], inplace=True)

c:\Users\cbarg\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
merged_df = pd.merge(df_home, df_away, how='left', left_on=['date', 'opponent'], right_on=['date', 'team'])
merged_df.drop(columns=['opponent_x', 'opponent_y'], inplace=True)
merged_df.head()

,team_x,date,points_x,goals_x,goals_against_x,power_play_x,penalty_kill_x,shots_x,shots_against_x,FOWp_x,team_y,points_y,goals_y,goals_against_y,power_play_y,penalty_kill_y,shots_y,shots_against_y,FOWp_y
0,New Jersey Devils,2005-10-05,2,5,1,25.0,90.9,37.0,37.0,57.1,Pittsburgh Penguins,0,1,5,9.1,75.0,37.0,37.0,42.9
1,Philadelphia Flyers,2005-10-05,0,3,5,10.0,50.0,27.0,27.0,48.4,New York Rangers,2,5,3,50.0,90.0,27.0,27.0,51.6
2,Boston Bruins,2005-10-05,0,1,2,0.0,80.0,30.0,21.0,56.5,Montréal Canadiens,2,2,1,20.0,100.0,21.0,30.0,43.6
3,Buffalo Sabres,2005-10-05,2,6,4,18.2,85.7,39.0,26.0,48.5,New York Islanders,0,4,6,14.3,81.8,26.0,39.0,51.5
4,Toronto Maple Leafs,2005-10-05,1,2,2,9.1,100.0,25.0,23.0,47.5,Ottawa Senators,2,2,2,0.0,90.9,23.0,25.0,52.5


How to calculate rolling stats with a window of 5 shown here.

In [10]:
df_devils = df[df.team == 'New Jersey Devils']
df_devils['rolling_points_5'] = df_devils['points'].rolling(5).mean().shift()
df_devils['rolling_points_10'] = df_devils['points'].rolling(10).mean().shift()
df_devils['rolling_points_20'] = df_devils['points'].rolling(20).mean().shift()
df_devils['rolling_goals_5'] = df_devils['goals'].rolling(5).mean().shift()
df_devils['rolling_goals_10'] = df_devils['goals'].rolling(10).mean().shift()
df_devils['rolling_ga_5'] = df_devils['goals_against'].rolling(5).mean().shift()
df_devils['rolling_ga_10'] = df_devils['goals_against'].rolling(10).mean().shift()
df_devils['rolling_pp_5'] = df_devils['power_play'].rolling(5).mean().shift()
df_devils['rolling_pk_5'] = df_devils['penalty_kill'].rolling(5).mean().shift()
df_devils['rolling_shots_5'] = df_devils['shots'].rolling(5).mean().shift()
df_devils['rolling_sa_5'] = df_devils['shots_against'].rolling(5).mean().shift()
df_devils['rolling_fow_5'] = df_devils['FOWp'].rolling(5).mean().shift()
df_devils.to_csv('.//Team Frames//devils.csv', index=False)

df_penguins = df[df.team == 'Pittsburgh Penguins']
df_penguins['rolling_points_5'] = df_penguins['points'].rolling(5).mean().shift()
df_penguins['rolling_points_10'] = df_penguins['points'].rolling(10).mean().shift()
df_penguins['rolling_points_20'] = df_penguins['points'].rolling(20).mean().shift()
df_penguins['rolling_goals_5'] = df_penguins['goals'].rolling(5).mean().shift()
df_penguins['rolling_goals_10'] = df_penguins['goals'].rolling(10).mean().shift()
df_penguins['rolling_ga_5'] = df_penguins['goals_against'].rolling(5).mean().shift()
df_penguins['rolling_ga_10'] = df_penguins['goals_against'].rolling(10).mean().shift()
df_penguins['rolling_pp_5'] = df_penguins['power_play'].rolling(5).mean().shift()
df_penguins['rolling_pk_5'] = df_devils['penalty_kill'].rolling(5).mean().shift()
df_penguins['rolling_shots_5'] = df_penguins['shots'].rolling(5).mean().shift()
df_penguins['rolling_sa_5'] = df_penguins['shots_against'].rolling(5).mean().shift()
df_penguins['rolling_fow_5'] = df_penguins['FOWp'].rolling(5).mean().shift()
df_penguins.to_csv('.//Team Frames//penguins.csv', index=False)

df_flyers = df[df.team == 'Philadelphia Flyers']
df_flyers['rolling_points_5'] = df_flyers['points'].rolling(5).mean().shift()
df_flyers['rolling_points_10'] = df_flyers['points'].rolling(10).mean().shift()
df_flyers['rolling_points_20'] = df_flyers['points'].rolling(20).mean().shift()
df_flyers['rolling_goals_5'] = df_flyers['goals'].rolling(5).mean().shift()
df_flyers['rolling_goals_10'] = df_flyers['goals'].rolling(10).mean().shift()
df_flyers['rolling_ga_5'] = df_flyers['goals_against'].rolling(5).mean().shift()
df_flyers['rolling_ga_10'] = df_flyers['goals_against'].rolling(10).mean().shift()
df_flyers['rolling_pp_5'] = df_flyers['power_play'].rolling(5).mean().shift()
df_flyers['rolling_pk_5'] = df_flyers['penalty_kill'].rolling(5).mean().shift()
df_flyers['rolling_shots_5'] = df_flyers['shots'].rolling(5).mean().shift()
df_flyers['rolling_sa_5'] = df_flyers['shots_against'].rolling(5).mean().shift()
df_flyers['rolling_fow_5'] = df_flyers['FOWp'].rolling(5).mean().shift()
df_flyers.to_csv('.//Team Frames//flyers.csv', index=False)

df_leafs = df[df.team == 'Toronto Maple Leafs']
df_leafs['rolling_points_5'] = df_leafs['points'].rolling(5).mean().shift()
df_leafs['rolling_points_10'] = df_leafs['points'].rolling(10).mean().shift()
df_leafs['rolling_points_20'] = df_leafs['points'].rolling(20).mean().shift()
df_leafs['rolling_goals_5'] = df_leafs['goals'].rolling(5).mean().shift()
df_leafs['rolling_goals_10'] = df_leafs['goals'].rolling(10).mean().shift()
df_leafs['rolling_ga_5'] = df_leafs['goals_against'].rolling(5).mean().shift()
df_leafs['rolling_ga_10'] = df_leafs['goals_against'].rolling(10).mean().shift()
df_leafs['rolling_pp_5'] = df_leafs['power_play'].rolling(5).mean().shift()
df_leafs['rolling_pk_5'] = df_leafs['penalty_kill'].rolling(5).mean().shift()
df_leafs['rolling_shots_5'] = df_leafs['shots'].rolling(5).mean().shift()
df_leafs['rolling_sa_5'] = df_leafs['shots_against'].rolling(5).mean().shift()
df_leafs['rolling_fow_5'] = df_leafs['FOWp'].rolling(5).mean().shift()
df_leafs.to_csv('.//Team Frames//leafs.csv', index=False)

df_canadiens = df[df.team == 'Montréal Canadiens']
df_canadiens['rolling_points_5'] = df_canadiens['points'].rolling(5).mean().shift()
df_canadiens['rolling_points_10'] = df_canadiens['points'].rolling(10).mean().shift()
df_canadiens['rolling_points_20'] = df_canadiens['points'].rolling(20).mean().shift()
df_canadiens['rolling_goals_5'] = df_canadiens['goals'].rolling(5).mean().shift()
df_canadiens['rolling_goals_10'] = df_canadiens['goals'].rolling(10).mean().shift()
df_canadiens['rolling_ga_5'] = df_canadiens['goals_against'].rolling(5).mean().shift()
df_canadiens['rolling_ga_10'] = df_canadiens['goals_against'].rolling(10).mean().shift()
df_canadiens['rolling_pp_5'] = df_canadiens['power_play'].rolling(5).mean().shift()
df_canadiens['rolling_pk_5'] = df_canadiens['penalty_kill'].rolling(5).mean().shift()
df_canadiens['rolling_shots_5'] = df_canadiens['shots'].rolling(5).mean().shift()
df_canadiens['rolling_sa_5'] = df_canadiens['shots_against'].rolling(5).mean().shift()
df_canadiens['rolling_fow_5'] = df_canadiens['FOWp'].rolling(5).mean().shift()
df_canadiens.to_csv('.//Team Frames//canadiens.csv', index=False)

df_wild = df[df.team == 'Minnesota Wild']
df_wild['rolling_points_5'] = df_wild['points'].rolling(5).mean().shift()
df_wild['rolling_points_10'] = df_wild['points'].rolling(10).mean().shift()
df_wild['rolling_points_20'] = df_wild['points'].rolling(20).mean().shift()
df_wild['rolling_goals_5'] = df_wild['goals'].rolling(5).mean().shift()
df_wild['rolling_goals_10'] = df_wild['goals'].rolling(10).mean().shift()
df_wild['rolling_ga_5'] = df_wild['goals_against'].rolling(5).mean().shift()
df_wild['rolling_ga_10'] = df_wild['goals_against'].rolling(10).mean().shift()
df_wild['rolling_pp_5'] = df_wild['power_play'].rolling(5).mean().shift()
df_wild['rolling_pk_5'] = df_wild['penalty_kill'].rolling(5).mean().shift()
df_wild['rolling_shots_5'] = df_wild['shots'].rolling(5).mean().shift()
df_wild['rolling_sa_5'] = df_wild['shots_against'].rolling(5).mean().shift()
df_wild['rolling_fow_5'] = df_wild['FOWp'].rolling(5).mean().shift()
df_wild.to_csv('.//Team Frames//wild.csv', index=False)

df_avalanche = df[df.team == 'Colorado Avalanche']
df_avalanche['rolling_points_5'] = df_avalanche['points'].rolling(5).mean().shift()
df_avalanche['rolling_points_10'] = df_avalanche['points'].rolling(10).mean().shift()
df_avalanche['rolling_points_20'] = df_avalanche['points'].rolling(20).mean().shift()
df_avalanche['rolling_goals_5'] = df_avalanche['goals'].rolling(5).mean().shift()
df_avalanche['rolling_goals_10'] = df_avalanche['goals'].rolling(10).mean().shift()
df_avalanche['rolling_ga_5'] = df_avalanche['goals_against'].rolling(5).mean().shift()
df_avalanche['rolling_ga_10'] = df_avalanche['goals_against'].rolling(10).mean().shift()
df_avalanche['rolling_pp_5'] = df_avalanche['power_play'].rolling(5).mean().shift()
df_avalanche['rolling_pk_5'] = df_avalanche['penalty_kill'].rolling(5).mean().shift()
df_avalanche['rolling_shots_5'] = df_avalanche['shots'].rolling(5).mean().shift()
df_avalanche['rolling_sa_5'] = df_avalanche['shots_against'].rolling(5).mean().shift()
df_avalanche['rolling_fow_5'] = df_avalanche['FOWp'].rolling(5).mean().shift()
df_avalanche.to_csv('.//Team Frames//avalanche.csv', index=False)

df_oilers = df[df.team == 'Edmonton Oilers']
df_oilers['rolling_points_5'] = df_oilers['points'].rolling(5).mean().shift()
df_oilers['rolling_points_10'] = df_oilers['points'].rolling(10).mean().shift()
df_oilers['rolling_points_20'] = df_oilers['points'].rolling(20).mean().shift()
df_oilers['rolling_goals_5'] = df_oilers['goals'].rolling(5).mean().shift()
df_oilers['rolling_goals_10'] = df_oilers['goals'].rolling(10).mean().shift()
df_oilers['rolling_ga_5'] = df_oilers['goals_against'].rolling(5).mean().shift()
df_oilers['rolling_ga_10'] = df_oilers['goals_against'].rolling(10).mean().shift()
df_oilers['rolling_pp_5'] = df_oilers['power_play'].rolling(5).mean().shift()
df_oilers['rolling_pk_5'] = df_oilers['penalty_kill'].rolling(5).mean().shift()
df_oilers['rolling_shots_5'] = df_oilers['shots'].rolling(5).mean().shift()
df_oilers['rolling_sa_5'] = df_oilers['shots_against'].rolling(5).mean().shift()
df_oilers['rolling_fow_5'] = df_oilers['FOWp'].rolling(5).mean().shift()
df_oilers.to_csv('.//Team Frames//oilers.csv', index=False)

df_blackhawks = df[df.team == 'Chicago Blackhawks']
df_blackhawks['rolling_points_5'] = df_blackhawks['points'].rolling(5).mean().shift()
df_blackhawks['rolling_points_10'] = df_blackhawks['points'].rolling(10).mean().shift()
df_blackhawks['rolling_points_20'] = df_blackhawks['points'].rolling(20).mean().shift()
df_blackhawks['rolling_goals_5'] = df_blackhawks['goals'].rolling(5).mean().shift()
df_blackhawks['rolling_goals_10'] = df_blackhawks['goals'].rolling(10).mean().shift()
df_blackhawks['rolling_ga_5'] = df_blackhawks['goals_against'].rolling(5).mean().shift()
df_blackhawks['rolling_ga_10'] = df_blackhawks['goals_against'].rolling(10).mean().shift()
df_blackhawks['rolling_pp_5'] = df_blackhawks['power_play'].rolling(5).mean().shift()
df_blackhawks['rolling_pk_5'] = df_blackhawks['penalty_kill'].rolling(5).mean().shift()
df_blackhawks['rolling_shots_5'] = df_blackhawks['shots'].rolling(5).mean().shift()
df_blackhawks['rolling_sa_5'] = df_blackhawks['shots_against'].rolling(5).mean().shift()
df_blackhawks['rolling_fow_5'] = df_blackhawks['FOWp'].rolling(5).mean().shift()
df_blackhawks.to_csv('.//Team Frames//blackhawks.csv', index=False)

df_flames = df[df.team == 'Calgary Flames']
df_flames['rolling_points_5'] = df_flames['points'].rolling(5).mean().shift()
df_flames['rolling_points_10'] = df_flames['points'].rolling(10).mean().shift()
df_flames['rolling_points_20'] = df_flames['points'].rolling(20).mean().shift()
df_flames['rolling_goals_5'] = df_flames['goals'].rolling(5).mean().shift()
df_flames['rolling_goals_10'] = df_flames['goals'].rolling(10).mean().shift()
df_flames['rolling_ga_5'] = df_flames['goals_against'].rolling(5).mean().shift()
df_flames['rolling_ga_10'] = df_flames['goals_against'].rolling(10).mean().shift()
df_flames['rolling_pp_5'] = df_flames['power_play'].rolling(5).mean().shift()
df_flames['rolling_pk_5'] = df_flames['penalty_kill'].rolling(5).mean().shift()
df_flames['rolling_shots_5'] = df_flames['shots'].rolling(5).mean().shift()
df_flames['rolling_sa_5'] = df_flames['shots_against'].rolling(5).mean().shift()
df_flames['rolling_fow_5'] = df_flames['FOWp'].rolling(5).mean().shift()
df_flames.to_csv('.//Team Frames//flames.csv', index=False)

df_hurricanes = df[df.team == 'Carolina Hurricanes']
df_hurricanes['rolling_points_5'] = df_hurricanes['points'].rolling(5).mean().shift()
df_hurricanes['rolling_points_10'] = df_hurricanes['points'].rolling(10).mean().shift()
df_hurricanes['rolling_points_20'] = df_hurricanes['points'].rolling(20).mean().shift()
df_hurricanes['rolling_goals_5'] = df_hurricanes['goals'].rolling(5).mean().shift()
df_hurricanes['rolling_goals_10'] = df_hurricanes['goals'].rolling(10).mean().shift()
df_hurricanes['rolling_ga_5'] = df_hurricanes['goals_against'].rolling(5).mean().shift()
df_hurricanes['rolling_ga_10'] = df_hurricanes['goals_against'].rolling(10).mean().shift()
df_hurricanes['rolling_pp_5'] = df_hurricanes['power_play'].rolling(5).mean().shift()
df_hurricanes['rolling_pk_5'] = df_hurricanes['penalty_kill'].rolling(5).mean().shift()
df_hurricanes['rolling_shots_5'] = df_hurricanes['shots'].rolling(5).mean().shift()
df_hurricanes['rolling_sa_5'] = df_hurricanes['shots_against'].rolling(5).mean().shift()
df_hurricanes['rolling_fow_5'] = df_hurricanes['FOWp'].rolling(5).mean().shift()
df_hurricanes.to_csv('.//Team Frames//hurricanes.csv', index=False)

df_stars = df[df.team == 'Dallas Stars']
df_stars['rolling_points_5'] = df_stars['points'].rolling(5).mean().shift()
df_stars['rolling_points_10'] = df_stars['points'].rolling(10).mean().shift()
df_stars['rolling_points_20'] = df_stars['points'].rolling(20).mean().shift()
df_stars['rolling_goals_5'] = df_stars['goals'].rolling(5).mean().shift()
df_stars['rolling_goals_10'] = df_stars['goals'].rolling(10).mean().shift()
df_stars['rolling_ga_5'] = df_stars['goals_against'].rolling(5).mean().shift()
df_stars['rolling_ga_10'] = df_stars['goals_against'].rolling(10).mean().shift()
df_stars['rolling_pp_5'] = df_stars['power_play'].rolling(5).mean().shift()
df_stars['rolling_pk_5'] = df_stars['penalty_kill'].rolling(5).mean().shift()
df_stars['rolling_shots_5'] = df_stars['shots'].rolling(5).mean().shift()
df_stars['rolling_sa_5'] = df_stars['shots_against'].rolling(5).mean().shift()
df_stars['rolling_fow_5'] = df_stars['FOWp'].rolling(5).mean().shift()
df_stars.to_csv('.//Team Frames//stars.csv', index=False)

df_senators = df[df.team == 'Ottawa Senators']
df_senators['rolling_points_5'] = df_senators['points'].rolling(5).mean().shift()
df_senators['rolling_points_10'] = df_senators['points'].rolling(10).mean().shift()
df_senators['rolling_points_20'] = df_senators['points'].rolling(20).mean().shift()
df_senators['rolling_goals_5'] = df_senators['goals'].rolling(5).mean().shift()
df_senators['rolling_goals_10'] = df_senators['goals'].rolling(10).mean().shift()
df_senators['rolling_ga_5'] = df_senators['goals_against'].rolling(5).mean().shift()
df_senators['rolling_ga_10'] = df_senators['goals_against'].rolling(10).mean().shift()
df_senators['rolling_pp_5'] = df_senators['power_play'].rolling(5).mean().shift()
df_senators['rolling_pk_5'] = df_senators['penalty_kill'].rolling(5).mean().shift()
df_senators['rolling_shots_5'] = df_senators['shots'].rolling(5).mean().shift()
df_senators['rolling_sa_5'] = df_senators['shots_against'].rolling(5).mean().shift()
df_senators['rolling_fow_5'] = df_senators['FOWp'].rolling(5).mean().shift()
df_senators.to_csv('.//Team Frames//senators.csv', index=False)

df_sabres = df[df.team == 'Buffalo Sabres']
df_sabres['rolling_points_5'] = df_sabres['points'].rolling(5).mean().shift()
df_sabres['rolling_points_10'] = df_sabres['points'].rolling(10).mean().shift()
df_sabres['rolling_points_20'] = df_sabres['points'].rolling(20).mean().shift()
df_sabres['rolling_goals_5'] = df_sabres['goals'].rolling(5).mean().shift()
df_sabres['rolling_goals_10'] = df_sabres['goals'].rolling(10).mean().shift()
df_sabres['rolling_ga_5'] = df_sabres['goals_against'].rolling(5).mean().shift()
df_sabres['rolling_ga_10'] = df_sabres['goals_against'].rolling(10).mean().shift()
df_sabres['rolling_pp_5'] = df_sabres['power_play'].rolling(5).mean().shift()
df_sabres['rolling_pk_5'] = df_sabres['penalty_kill'].rolling(5).mean().shift()
df_sabres['rolling_shots_5'] = df_sabres['shots'].rolling(5).mean().shift()
df_sabres['rolling_sa_5'] = df_sabres['shots_against'].rolling(5).mean().shift()
df_sabres['rolling_fow_5'] = df_sabres['FOWp'].rolling(5).mean().shift()
df_sabres.to_csv('.//Team Frames//sabres.csv', index=False)

df_capitals = df[df.team == 'Washington Capitals']
df_capitals['rolling_points_5'] = df_capitals['points'].rolling(5).mean().shift()
df_capitals['rolling_points_10'] = df_capitals['points'].rolling(10).mean().shift()
df_capitals['rolling_points_20'] = df_capitals['points'].rolling(20).mean().shift()
df_capitals['rolling_goals_5'] = df_capitals['goals'].rolling(5).mean().shift()
df_capitals['rolling_goals_10'] = df_capitals['goals'].rolling(10).mean().shift()
df_capitals['rolling_ga_5'] = df_capitals['goals_against'].rolling(5).mean().shift()
df_capitals['rolling_ga_10'] = df_capitals['goals_against'].rolling(10).mean().shift()
df_capitals['rolling_pp_5'] = df_capitals['power_play'].rolling(5).mean().shift()
df_capitals['rolling_pk_5'] = df_capitals['penalty_kill'].rolling(5).mean().shift()
df_capitals['rolling_shots_5'] = df_capitals['shots'].rolling(5).mean().shift()
df_capitals['rolling_sa_5'] = df_capitals['shots_against'].rolling(5).mean().shift()
df_capitals['rolling_fow_5'] = df_capitals['FOWp'].rolling(5).mean().shift()
df_capitals.to_csv('.//Team Frames//capitals.csv', index=False)

df_thrashers = df[df.team == 'Atlanta Thrashers']
df_thrashers['rolling_points_5'] = df_thrashers['points'].rolling(5).mean().shift()
df_thrashers['rolling_points_10'] = df_thrashers['points'].rolling(10).mean().shift()
df_thrashers['rolling_points_20'] = df_thrashers['points'].rolling(20).mean().shift()
df_thrashers['rolling_goals_5'] = df_thrashers['goals'].rolling(5).mean().shift()
df_thrashers['rolling_goals_10'] = df_thrashers['goals'].rolling(10).mean().shift()
df_thrashers['rolling_ga_5'] = df_thrashers['goals_against'].rolling(5).mean().shift()
df_thrashers['rolling_ga_10'] = df_thrashers['goals_against'].rolling(10).mean().shift()
df_thrashers['rolling_pp_5'] = df_thrashers['power_play'].rolling(5).mean().shift()
df_thrashers['rolling_pk_5'] = df_thrashers['penalty_kill'].rolling(5).mean().shift()
df_thrashers['rolling_shots_5'] = df_thrashers['shots'].rolling(5).mean().shift()
df_thrashers['rolling_sa_5'] = df_thrashers['shots_against'].rolling(5).mean().shift()
df_thrashers['rolling_fow_5'] = df_thrashers['FOWp'].rolling(5).mean().shift()
df_thrashers.to_csv('.//Team Frames//thrashers.csv', index=False)

df_ducks = df[df.team == 'Anaheim Ducks']
df_ducks['rolling_points_5'] = df_ducks['points'].rolling(5).mean().shift()
df_ducks['rolling_points_10'] = df_ducks['points'].rolling(10).mean().shift()
df_ducks['rolling_points_20'] = df_ducks['points'].rolling(20).mean().shift()
df_ducks['rolling_goals_5'] = df_ducks['goals'].rolling(5).mean().shift()
df_ducks['rolling_goals_10'] = df_ducks['goals'].rolling(10).mean().shift()
df_ducks['rolling_ga_5'] = df_ducks['goals_against'].rolling(5).mean().shift()
df_ducks['rolling_ga_10'] = df_ducks['goals_against'].rolling(10).mean().shift()
df_ducks['rolling_pp_5'] = df_ducks['power_play'].rolling(5).mean().shift()
df_ducks['rolling_pk_5'] = df_ducks['penalty_kill'].rolling(5).mean().shift()
df_ducks['rolling_shots_5'] = df_ducks['shots'].rolling(5).mean().shift()
df_ducks['rolling_sa_5'] = df_ducks['shots_against'].rolling(5).mean().shift()
df_ducks['rolling_fow_5'] = df_ducks['FOWp'].rolling(5).mean().shift()
df_ducks.to_csv('.//Team Frames//ducks.csv', index=False)

df_redwings = df[df.team == 'Detroit Red Wings']
df_redwings['rolling_points_5'] = df_redwings['points'].rolling(5).mean().shift()
df_redwings['rolling_points_10'] = df_redwings['points'].rolling(10).mean().shift()
df_redwings['rolling_points_20'] = df_redwings['points'].rolling(20).mean().shift()
df_redwings['rolling_goals_5'] = df_redwings['goals'].rolling(5).mean().shift()
df_redwings['rolling_goals_10'] = df_redwings['goals'].rolling(10).mean().shift()
df_redwings['rolling_ga_5'] = df_redwings['goals_against'].rolling(5).mean().shift()
df_redwings['rolling_ga_10'] = df_redwings['goals_against'].rolling(10).mean().shift()
df_redwings['rolling_pp_5'] = df_redwings['power_play'].rolling(5).mean().shift()
df_redwings['rolling_pk_5'] = df_redwings['penalty_kill'].rolling(5).mean().shift()
df_redwings['rolling_shots_5'] = df_redwings['shots'].rolling(5).mean().shift()
df_redwings['rolling_sa_5'] = df_redwings['shots_against'].rolling(5).mean().shift()
df_redwings['rolling_fow_5'] = df_redwings['FOWp'].rolling(5).mean().shift()
df_redwings.to_csv('.//Team Frames//redwings.csv', index=False)

df_bluejackets = df[df.team == 'Columbus Blue Jackets']
df_bluejackets['rolling_points_5'] = df_bluejackets['points'].rolling(5).mean().shift()
df_bluejackets['rolling_points_10'] = df_bluejackets['points'].rolling(10).mean().shift()
df_bluejackets['rolling_points_20'] = df_bluejackets['points'].rolling(20).mean().shift()
df_bluejackets['rolling_goals_5'] = df_bluejackets['goals'].rolling(5).mean().shift()
df_bluejackets['rolling_goals_10'] = df_bluejackets['goals'].rolling(10).mean().shift()
df_bluejackets['rolling_ga_5'] = df_bluejackets['goals_against'].rolling(5).mean().shift()
df_bluejackets['rolling_ga_10'] = df_bluejackets['goals_against'].rolling(10).mean().shift()
df_bluejackets['rolling_pp_5'] = df_bluejackets['power_play'].rolling(5).mean().shift()
df_bluejackets['rolling_pk_5'] = df_bluejackets['penalty_kill'].rolling(5).mean().shift()
df_bluejackets['rolling_shots_5'] = df_bluejackets['shots'].rolling(5).mean().shift()
df_bluejackets['rolling_sa_5'] = df_bluejackets['shots_against'].rolling(5).mean().shift()
df_bluejackets['rolling_fow_5'] = df_senators['FOWp'].rolling(5).mean().shift()
df_bluejackets.to_csv('.//Team Frames//bluejackets.csv', index=False)

df_sharks = df[df.team == 'San Jose Sharks']
df_sharks['rolling_points_5'] = df_sharks['points'].rolling(5).mean().shift()
df_sharks['rolling_points_10'] = df_sharks['points'].rolling(10).mean().shift()
df_sharks['rolling_points_20'] = df_sharks['points'].rolling(20).mean().shift()
df_sharks['rolling_goals_5'] = df_sharks['goals'].rolling(5).mean().shift()
df_sharks['rolling_goals_10'] = df_sharks['goals'].rolling(10).mean().shift()
df_sharks['rolling_ga_5'] = df_sharks['goals_against'].rolling(5).mean().shift()
df_sharks['rolling_ga_10'] = df_sharks['goals_against'].rolling(10).mean().shift()
df_sharks['rolling_pp_5'] = df_sharks['power_play'].rolling(5).mean().shift()
df_sharks['rolling_pk_5'] = df_sharks['penalty_kill'].rolling(5).mean().shift()
df_sharks['rolling_shots_5'] = df_sharks['shots'].rolling(5).mean().shift()
df_sharks['rolling_sa_5'] = df_sharks['shots_against'].rolling(5).mean().shift()
df_sharks['rolling_fow_5'] = df_sharks['FOWp'].rolling(5).mean().shift()
df_sharks.to_csv('.//Team Frames//sharks.csv', index=False)

df_blues = df[df.team == 'St. Louis Blues']
df_blues['rolling_points_5'] = df_blues['points'].rolling(5).mean().shift()
df_blues['rolling_points_10'] = df_blues['points'].rolling(10).mean().shift()
df_blues['rolling_points_20'] = df_blues['points'].rolling(20).mean().shift()
df_blues['rolling_goals_5'] = df_blues['goals'].rolling(5).mean().shift()
df_blues['rolling_goals_10'] = df_blues['goals'].rolling(10).mean().shift()
df_blues['rolling_ga_5'] = df_blues['goals_against'].rolling(5).mean().shift()
df_blues['rolling_ga_10'] = df_blues['goals_against'].rolling(10).mean().shift()
df_blues['rolling_pp_5'] = df_blues['power_play'].rolling(5).mean().shift()
df_blues['rolling_pk_5'] = df_blues['penalty_kill'].rolling(5).mean().shift()
df_blues['rolling_shots_5'] = df_blues['shots'].rolling(5).mean().shift()
df_blues['rolling_sa_5'] = df_blues['shots_against'].rolling(5).mean().shift()
df_blues['rolling_fow_5'] = df_blues['FOWp'].rolling(5).mean().shift()
df_blues.to_csv('.//Team Frames//blues.csv', index=False)

df_coyotes = df[(df.team == 'Phoenix Coyotes') | (df.team == 'Arizona Coyotes')]
df_coyotes['rolling_points_5'] = df_coyotes['points'].rolling(5).mean().shift()
df_coyotes['rolling_points_10'] = df_coyotes['points'].rolling(10).mean().shift()
df_coyotes['rolling_points_20'] = df_coyotes['points'].rolling(20).mean().shift()
df_coyotes['rolling_goals_5'] = df_coyotes['goals'].rolling(5).mean().shift()
df_coyotes['rolling_goals_10'] = df_coyotes['goals'].rolling(10).mean().shift()
df_coyotes['rolling_ga_5'] = df_coyotes['goals_against'].rolling(5).mean().shift()
df_coyotes['rolling_ga_10'] = df_coyotes['goals_against'].rolling(10).mean().shift()
df_coyotes['rolling_pp_5'] = df_coyotes['power_play'].rolling(5).mean().shift()
df_coyotes['rolling_pk_5'] = df_coyotes['penalty_kill'].rolling(5).mean().shift()
df_coyotes['rolling_shots_5'] = df_coyotes['shots'].rolling(5).mean().shift()
df_coyotes['rolling_sa_5'] = df_coyotes['shots_against'].rolling(5).mean().shift()
df_coyotes['rolling_fow_5'] = df_coyotes['FOWp'].rolling(5).mean().shift()
df_coyotes.to_csv('.//Team Frames//coyotes.csv', index=False)

df_rangers = df[df.team == 'New York Rangers']
df_rangers['rolling_points_5'] = df_rangers['points'].rolling(5).mean().shift()
df_rangers['rolling_points_10'] = df_rangers['points'].rolling(10).mean().shift()
df_rangers['rolling_points_20'] = df_rangers['points'].rolling(20).mean().shift()
df_rangers['rolling_goals_5'] = df_rangers['goals'].rolling(5).mean().shift()
df_rangers['rolling_goals_10'] = df_rangers['goals'].rolling(10).mean().shift()
df_rangers['rolling_ga_5'] = df_rangers['goals_against'].rolling(5).mean().shift()
df_rangers['rolling_ga_10'] = df_rangers['goals_against'].rolling(10).mean().shift()
df_rangers['rolling_pp_5'] = df_rangers['power_play'].rolling(5).mean().shift()
df_rangers['rolling_pk_5'] = df_rangers['penalty_kill'].rolling(5).mean().shift()
df_rangers['rolling_shots_5'] = df_rangers['shots'].rolling(5).mean().shift()
df_rangers['rolling_sa_5'] = df_rangers['shots_against'].rolling(5).mean().shift()
df_rangers['rolling_fow_5'] = df_rangers['FOWp'].rolling(5).mean().shift()
df_rangers.to_csv('.//Team Frames//rangers.csv', index=False)

df_lightning = df[df.team == 'Tampa Bay Lightning']
df_lightning['rolling_points_5'] = df_lightning['points'].rolling(5).mean().shift()
df_lightning['rolling_points_10'] = df_lightning['points'].rolling(10).mean().shift()
df_lightning['rolling_points_20'] = df_lightning['points'].rolling(20).mean().shift()
df_lightning['rolling_goals_5'] = df_lightning['goals'].rolling(5).mean().shift()
df_lightning['rolling_goals_10'] = df_lightning['goals'].rolling(10).mean().shift()
df_lightning['rolling_ga_5'] = df_lightning['goals_against'].rolling(5).mean().shift()
df_lightning['rolling_ga_10'] = df_lightning['goals_against'].rolling(10).mean().shift()
df_lightning['rolling_pp_5'] = df_lightning['power_play'].rolling(5).mean().shift()
df_lightning['rolling_pk_5'] = df_lightning['penalty_kill'].rolling(5).mean().shift()
df_lightning['rolling_shots_5'] = df_lightning['shots'].rolling(5).mean().shift()
df_lightning['rolling_sa_5'] = df_lightning['shots_against'].rolling(5).mean().shift()
df_lightning['rolling_fow_5'] = df_lightning['FOWp'].rolling(5).mean().shift()
df_lightning.to_csv('.//Team Frames//lightning.csv', index=False)

df_kings = df[df.team == 'Los Angeles Kings']
df_kings['rolling_points_5'] = df_kings['points'].rolling(5).mean().shift()
df_kings['rolling_points_10'] = df_kings['points'].rolling(10).mean().shift()
df_kings['rolling_points_20'] = df_kings['points'].rolling(20).mean().shift()
df_kings['rolling_goals_5'] = df_kings['goals'].rolling(5).mean().shift()
df_kings['rolling_goals_10'] = df_kings['goals'].rolling(10).mean().shift()
df_kings['rolling_ga_5'] = df_kings['goals_against'].rolling(5).mean().shift()
df_kings['rolling_ga_10'] = df_kings['goals_against'].rolling(10).mean().shift()
df_kings['rolling_pp_5'] = df_kings['power_play'].rolling(5).mean().shift()
df_kings['rolling_pk_5'] = df_kings['penalty_kill'].rolling(5).mean().shift()
df_kings['rolling_shots_5'] = df_kings['shots'].rolling(5).mean().shift()
df_kings['rolling_sa_5'] = df_kings['shots_against'].rolling(5).mean().shift()
df_kings['rolling_fow_5'] = df_kings['FOWp'].rolling(5).mean().shift()
df_kings.to_csv('.//Team Frames//kings.csv', index=False)

df_predators = df[df.team == 'Nashville Predators']
df_predators['rolling_points_5'] = df_predators['points'].rolling(5).mean().shift()
df_predators['rolling_points_10'] = df_predators['points'].rolling(10).mean().shift()
df_predators['rolling_points_20'] = df_predators['points'].rolling(20).mean().shift()
df_predators['rolling_goals_5'] = df_predators['goals'].rolling(5).mean().shift()
df_predators['rolling_goals_10'] = df_predators['goals'].rolling(10).mean().shift()
df_predators['rolling_ga_5'] = df_predators['goals_against'].rolling(5).mean().shift()
df_predators['rolling_ga_10'] = df_predators['goals_against'].rolling(10).mean().shift()
df_predators['rolling_pp_5'] = df_predators['power_play'].rolling(5).mean().shift()
df_predators['rolling_pk_5'] = df_predators['penalty_kill'].rolling(5).mean().shift()
df_predators['rolling_shots_5'] = df_predators['shots'].rolling(5).mean().shift()
df_predators['rolling_sa_5'] = df_predators['shots_against'].rolling(5).mean().shift()
df_predators['rolling_fow_5'] = df_predators['FOWp'].rolling(5).mean().shift()
df_predators.to_csv('.//Team Frames//predators.csv', index=False)

df_islanders = df[df.team == 'New York Islanders']
df_islanders['rolling_points_5'] = df_islanders['points'].rolling(5).mean().shift()
df_islanders['rolling_points_10'] = df_islanders['points'].rolling(10).mean().shift()
df_islanders['rolling_points_20'] = df_islanders['points'].rolling(20).mean().shift()
df_islanders['rolling_goals_5'] = df_islanders['goals'].rolling(5).mean().shift()
df_islanders['rolling_goals_10'] = df_islanders['goals'].rolling(10).mean().shift()
df_islanders['rolling_ga_5'] = df_islanders['goals_against'].rolling(5).mean().shift()
df_islanders['rolling_ga_10'] = df_islanders['goals_against'].rolling(10).mean().shift()
df_islanders['rolling_pp_5'] = df_islanders['power_play'].rolling(5).mean().shift()
df_islanders['rolling_pk_5'] = df_islanders['penalty_kill'].rolling(5).mean().shift()
df_islanders['rolling_shots_5'] = df_islanders['shots'].rolling(5).mean().shift()
df_islanders['rolling_sa_5'] = df_islanders['shots_against'].rolling(5).mean().shift()
df_islanders['rolling_fow_5'] = df_islanders['FOWp'].rolling(5).mean().shift()
df_islanders.to_csv('.//Team Frames//islanders.csv', index=False)

df_canucks = df[df.team == 'Vancouver Canucks']
df_canucks['rolling_points_5'] = df_canucks['points'].rolling(5).mean().shift()
df_canucks['rolling_points_10'] = df_canucks['points'].rolling(10).mean().shift()
df_canucks['rolling_points_20'] = df_canucks['points'].rolling(20).mean().shift()
df_canucks['rolling_goals_5'] = df_canucks['goals'].rolling(5).mean().shift()
df_canucks['rolling_goals_10'] = df_canucks['goals'].rolling(10).mean().shift()
df_canucks['rolling_ga_5'] = df_canucks['goals_against'].rolling(5).mean().shift()
df_canucks['rolling_ga_10'] = df_canucks['goals_against'].rolling(10).mean().shift()
df_canucks['rolling_pp_5'] = df_canucks['power_play'].rolling(5).mean().shift()
df_canucks['rolling_pk_5'] = df_canucks['penalty_kill'].rolling(5).mean().shift()
df_canucks['rolling_shots_5'] = df_canucks['shots'].rolling(5).mean().shift()
df_canucks['rolling_sa_5'] = df_canucks['shots_against'].rolling(5).mean().shift()
df_canucks['rolling_fow_5'] = df_canucks['FOWp'].rolling(5).mean().shift()
df_canucks.to_csv('.//Team Frames//canucks.csv', index=False)

df_bruins = df[df.team == 'Boston Bruins']
df_bruins['rolling_points_5'] = df_bruins['points'].rolling(5).mean().shift()
df_bruins['rolling_points_10'] = df_bruins['points'].rolling(10).mean().shift()
df_bruins['rolling_points_20'] = df_bruins['points'].rolling(20).mean().shift()
df_bruins['rolling_goals_5'] = df_bruins['goals'].rolling(5).mean().shift()
df_bruins['rolling_goals_10'] = df_bruins['goals'].rolling(10).mean().shift()
df_bruins['rolling_ga_5'] = df_bruins['goals_against'].rolling(5).mean().shift()
df_bruins['rolling_ga_10'] = df_bruins['goals_against'].rolling(10).mean().shift()
df_bruins['rolling_pp_5'] = df_bruins['power_play'].rolling(5).mean().shift()
df_bruins['rolling_pk_5'] = df_bruins['penalty_kill'].rolling(5).mean().shift()
df_bruins['rolling_shots_5'] = df_bruins['shots'].rolling(5).mean().shift()
df_bruins['rolling_sa_5'] = df_bruins['shots_against'].rolling(5).mean().shift()
df_bruins['rolling_fow_5'] = df_bruins['FOWp'].rolling(5).mean().shift()
df_bruins.to_csv('.//Team Frames//bruins.csv', index=False)

df_panthers = df[df.team == 'Florida Panthers']
df_panthers['rolling_points_5'] = df_panthers['points'].rolling(5).mean().shift()
df_panthers['rolling_points_10'] = df_panthers['points'].rolling(10).mean().shift()
df_panthers['rolling_points_20'] = df_panthers['points'].rolling(20).mean().shift()
df_panthers['rolling_goals_5'] = df_panthers['goals'].rolling(5).mean().shift()
df_panthers['rolling_goals_10'] = df_panthers['goals'].rolling(10).mean().shift()
df_panthers['rolling_ga_5'] = df_panthers['goals_against'].rolling(5).mean().shift()
df_panthers['rolling_ga_10'] = df_panthers['goals_against'].rolling(10).mean().shift()
df_panthers['rolling_pp_5'] = df_panthers['power_play'].rolling(5).mean().shift()
df_panthers['rolling_pk_5'] = df_panthers['penalty_kill'].rolling(5).mean().shift()
df_panthers['rolling_shots_5'] = df_panthers['shots'].rolling(5).mean().shift()
df_panthers['rolling_sa_5'] = df_panthers['shots_against'].rolling(5).mean().shift()
df_panthers['rolling_fow_5'] = df_panthers['FOWp'].rolling(5).mean().shift()
df_panthers.to_csv('.//Team Frames//panthers.csv', index=False)

df_jets = df[df.team == 'Winnipeg Jets']
df_jets['rolling_points_5'] = df_jets['points'].rolling(5).mean().shift()
df_jets['rolling_points_10'] = df_jets['points'].rolling(10).mean().shift()
df_jets['rolling_points_20'] = df_jets['points'].rolling(20).mean().shift()
df_jets['rolling_goals_5'] = df_jets['goals'].rolling(5).mean().shift()
df_jets['rolling_goals_10'] = df_jets['goals'].rolling(10).mean().shift()
df_jets['rolling_ga_5'] = df_jets['goals_against'].rolling(5).mean().shift()
df_jets['rolling_ga_10'] = df_jets['goals_against'].rolling(10).mean().shift()
df_jets['rolling_pp_5'] = df_jets['power_play'].rolling(5).mean().shift()
df_jets['rolling_pk_5'] = df_jets['penalty_kill'].rolling(5).mean().shift()
df_jets['rolling_shots_5'] = df_jets['shots'].rolling(5).mean().shift()
df_jets['rolling_sa_5'] = df_jets['shots_against'].rolling(5).mean().shift()
df_jets['rolling_fow_5'] = df_jets['FOWp'].rolling(5).mean().shift()
df_jets.to_csv('.//Team Frames//jets.csv', index=False)

df_goldenknights = df[df.team == 'Vegas Golden Knights']
df_goldenknights['rolling_points_5'] = df_goldenknights['points'].rolling(5).mean().shift()
df_goldenknights['rolling_points_10'] = df_goldenknights['points'].rolling(10).mean().shift()
df_goldenknights['rolling_points_20'] = df_goldenknights['points'].rolling(20).mean().shift()
df_goldenknights['rolling_goals_5'] = df_goldenknights['goals'].rolling(5).mean().shift()
df_goldenknights['rolling_goals_10'] = df_goldenknights['goals'].rolling(10).mean().shift()
df_goldenknights['rolling_ga_5'] = df_goldenknights['goals_against'].rolling(5).mean().shift()
df_goldenknights['rolling_ga_10'] = df_goldenknights['goals_against'].rolling(10).mean().shift()
df_goldenknights['rolling_pp_5'] = df_goldenknights['power_play'].rolling(5).mean().shift()
df_goldenknights['rolling_pk_5'] = df_goldenknights['penalty_kill'].rolling(5).mean().shift()
df_goldenknights['rolling_shots_5'] = df_goldenknights['shots'].rolling(5).mean().shift()
df_goldenknights['rolling_sa_5'] = df_goldenknights['shots_against'].rolling(5).mean().shift()
df_goldenknights['rolling_fow_5'] = df_goldenknights['FOWp'].rolling(5).mean().shift()
df_goldenknights.to_csv('.//Team Frames//goldenknights.csv', index=False)

df_kraken = df[df.team == 'Seattle Kraken']
df_kraken['rolling_points_5'] = df_kraken['points'].rolling(5).mean().shift()
df_kraken['rolling_points_10'] = df_kraken['points'].rolling(10).mean().shift()
df_kraken['rolling_points_20'] = df_kraken['points'].rolling(20).mean().shift()
df_kraken['rolling_goals_5'] = df_kraken['goals'].rolling(5).mean().shift()
df_kraken['rolling_goals_10'] = df_kraken['goals'].rolling(10).mean().shift()
df_kraken['rolling_ga_5'] = df_kraken['goals_against'].rolling(5).mean().shift()
df_kraken['rolling_ga_10'] = df_kraken['goals_against'].rolling(10).mean().shift()
df_kraken['rolling_pp_5'] = df_kraken['power_play'].rolling(5).mean().shift()
df_kraken['rolling_pk_5'] = df_kraken['penalty_kill'].rolling(5).mean().shift()
df_kraken['rolling_shots_5'] = df_kraken['shots'].rolling(5).mean().shift()
df_kraken['rolling_sa_5'] = df_kraken['shots_against'].rolling(5).mean().shift()
df_kraken['rolling_fow_5'] = df_kraken['FOWp'].rolling(5).mean().shift()
df_kraken.to_csv('.//Team Frames//kraken.csv', index=False)

merged_df_test = merged_df.copy()
df_devils.head(10)

abb_to_team = {'PIT':'Pittsburgh Penguins', 'PHI':'Philadelphia Flyers',
'TOR':'Toronto Maple Leafs', 'MTL':'Montréal Canadiens', 'MIN':'Minnesota Wild',
'COL':'Colorado Avalanche', 'EDM':'Edmonton Oilers', 'CHI':'Chicago Blackhawks',
'CGY':'Calgary Flames', 'CAR':'Carolina Hurricanes', 'DAL':'Dallas Stars',
'OTT':'Ottawa Senators', 'BUF':'Buffalo Sabres', 'WSH':'Washington Capitals',
'ATL':'Atlanta Thrashers', 'ANA':'Anaheim Ducks', 'DET':'Detroit Red Wings',
'NJD':'New Jersey Devils','CBJ':'Columbus Blue Jackets', 'SJS':'San Jose Sharks',
'STL':'St. Louis Blues', 'PHX':'Phoenix Coyotes', 'NYR':'New York Rangers',
'TBL':'Tampa Bay Lightning', 'LAK':'Los Angeles Kings', 'NSH':'Nashville Predators',
'NYI':'New York Islanders', 'VAN':'Vancouver Canucks', 'BOS':'Boston Bruins',
'FLA':'Florida Panthers', 'WPG':'Winnipeg Jets', 'ARI':'Arizona Coyotes',
'VGK':'Vegas Golden Knights', 'SEA':'Seattle Kraken'}


<ipython-input-10-7babc446bb00>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_devils['rolling_points_5'] = df_devils['points'].rolling(5).mean().shift()
<ipython-input-10-7babc446bb00>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_devils['rolling_points_10'] = df_devils['points'].rolling(10).mean().shift()
<ipython-input-10-7babc446bb00>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 